## Interface Supervisor Agent

In the previous notebooks, we built three specialized agents and a supervisor agent:

1. **Researach Agent** (Notebook 01): An expert at identifying entities in facts gathered from a news event, and then researching the background of those entities to produce data that can assist in writing a news story
2. **Article Generation Agent** (Notebook 02): An expert content creator that writes high-quality news, sports, and entertainment articles following detailed journalistic guidelines. This agent focuses on clarity, accuracy, engagement, structure, balance, and technical excellence.
3. **Article Reviewer Agent** (Notebook 03): A professional reviewer that analyzes articles across multiple dimensions including readability, factual accuracy, engagement, fairness, context, and ethical considerations. This agent provides detailed, constructive feedback with specific improvement suggestions.
4. **Article Supervisor Agent** (Notebook 03): A supervisor agent that orchestrates the article generator and reviewer specialized agents to work together iteratively. This pattern enables a powerful feedback loop where content is continuously refined through expert review cycles.

In this notebook, we'll implement an Interface Supervisor agent that applies a multi-agent collaboration pattern to communicate between the user, research agent, and article generation supervisor agent.

## Multi-Agent Collaboration Pattern

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is an Amazon Bedrock Agents capability that enables hierarchical collaboration between agents. This pattern allows:

- A supervisor agent to coordinate specialized sub-agents
- Delegation of specific tasks to agents with specialized expertise
- Iterative improvement through feedback loops
- Complex workflows that leverage multiple AI capabilities

The supervisor agent we'll create will:
1. Receive the initial news facts regarding a news event from the user
2. Delegates verification and research around entities identified in the news facts to the Research Agent
2. The output of the research agents is then sent to the Article Generation Supervisor, who uses the reflection pattern to iteratively produce a new story.

The following architecture diagram illustrates this multi-agent collaboration workflow:

<img src="../imgs/mac-news-generation.png" width="800">

Let's get started!

First we restore the variables from the previous notebook

In [ ]:
%store -r

## Create an Interface Supervisor agent using Amazon Bedrock Agents
On this section we declare global variables that will be act as helpers during entire notebook.

In [ ]:
import boto3
import time
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-premier-v1:0'
]


## Importing helper functions
On following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation and accessing data on S3.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Article Generation Supervisor Agent
Create a supervisor agent which will direct the users input to the Research Agent and then send all the information to the Aritcle Generation Supervisor Agent.


In [ ]:
interface_supervisor_agent_name = "interface_supervisor_agent"
agent_description = "interfaces between the user, research agent and article generation supervisor"

# In the following instructions note that we are referring to the two collaborate agents by their collaborator names, the collaborators will be added later
instruction = """You are a supervisor agent responsible for orchestrating a news article generation workflow.
You will be provided news facts that have been collected from a news event. 
Your role is to only coordinate between agents.

Here are your primary functions:
1. Delegate Research: Pass these news facts unmodified to the researchSupervisor agent. The unmodified output of the researchSupervisor agent should be sent to the generateSupervisor agent.
2. Delegate Article Generation: Take the complete output from the researchSupervisor agent and pass it unmodified to the generateSupervisor agent. 
3. Return final news article: Return the generateSupervisor agent's output to the user without any modifications.

Important Guidelines:
Do not modify, summarize, or filter the researchSupervisor agent's output before passing it to the generateSupervisor agent.
Do not edit, rewrite, or enhance the generateSupervisor agent's output before returning it to the user.
Maintain a clear separation between the two stages of the workflow.
If any agent returns an error or incomplete output, notify the user with the exact error message.

Your function is purely to orchestrate this workflow with complete fidelity to the inputs and outputs of each stage, without adding your own analysis or modifications."""

supervisor_agent = agents.create_agent(
    interface_supervisor_agent_name,
    agent_description,
    instruction,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR'
)

interface_supervisor_agent_id = supervisor_agent[0]
interface_supervisor_agent_alias_id = supervisor_agent[1]
interface_supervisor_agent_alias_arn = supervisor_agent[2]
agent_resourceRoleArn = supervisor_agent[3]

### Associate Collaborators

In this section, we're going to recover alias Ids of the following agents to add them as collaborators for the Interface Supervisor Agent:
1. Research Agent (Notebook 01)
2. Article Generation Supervisor (Notebook 05)

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': research_agent_alias_arn,
        'sub_agent_instruction': """The researchSupervisor can extract entities based on information from a news event, verify those entities exist, and then gather information about those entities.""",
        'sub_agent_association_name': 'researchSupervisor',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': article_supervisor_agent_alias_arn,
        'sub_agent_instruction': """The generateSupervisor agent can generate news articles for news websites given information about a news event.""",
        'sub_agent_association_name': 'generateSupervisor',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Associate all sub-agents with the Interface Supervisor Agent:

In [ ]:
interface_supervisor_agent_alias_id, interface_supervisor_agent_alias_arn = agents.associate_sub_agents(
    interface_supervisor_agent_id, sub_agents_list
)

We update the agent with new default temperature, topP, topK, and maximumLength values

Updating an agent requires setting a lot of values, in the cell below we grab the default values to feed them back into the `update_agent` function.

Read more about `update_agent` [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/update_agent.html) and about Advanced Prompts [here](https://docs.aws.amazon.com/bedrock/latest/userguide/advanced-prompts.html)

In [ ]:
# Sleep to let the agent versioning finish
time.sleep(20)

bedrock_agent_client = boto3.client('bedrock-agent')

# Helper function to find the right basepromptTemplate
def find_by_key_value_next(items, key, value):
    return next((item for item in items if item[key] == value), None)


# This helps us grab the correct basePromptTemplate used for the orchestration step
def get_base_prompt_template(promptType, agentId):
    # get all the info in the agent at the current state
    agent_info = bedrock_agent_client.get_agent(agentId=agentId)

    # Go through the results to find the info we need for update agent
    # You can see the full response of get_agent here:
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/get_agent.html
    prompt_orchestrations = agent_info['agent']['promptOverrideConfiguration']['promptConfigurations']
    return find_by_key_value_next(prompt_orchestrations,
                                  'promptType',
                                  promptType)['basePromptTemplate']


# We have to change the inference values only for the orchestration step
prompt_template = get_base_prompt_template('ORCHESTRATION',
                                           interface_supervisor_agent_id)

response = bedrock_agent_client.update_agent(
        agentCollaboration='SUPERVISOR',
        agentId=interface_supervisor_agent_id,
        agentName=interface_supervisor_agent_name,
        description=agent_description,
        agentResourceRoleArn=agent_resourceRoleArn,
        foundationModel=agent_foundation_model[0],
        instruction=instruction,
        promptOverrideConfiguration={
            "promptConfigurations": [
                {
                    "inferenceConfiguration": {
                        "temperature": 1.0,
                        "topP": 1.0,
                        "topK": 1,
                        "maximumLength": 8192
                    },
                    'parserMode': 'DEFAULT',
                    'promptCreationMode': 'OVERRIDDEN',
                    'promptType': 'ORCHESTRATION',
                    'promptState': 'ENABLED',
                    'basePromptTemplate': prompt_template
                }
            ]
        }
)

# An agent always has to be prepared after changes
bedrock_agent_client.prepare_agent(agentId=interface_supervisor_agent_id)

# Sleep to let the agent preparation finish
time.sleep(20)

## Invoking multi-agent Collaborator

With our supervisor agent created and connected to both specialized agents, we can now demonstrate the power of multi-agent collaboration. The supervisor agent will orchestrate the entire workflow:

1. It will first send the user input to the Research Agent for entity recognition, verification and additional research
2. The results of the research and the original news facts will then be sent to the Generation Supervisor Agent
3. The Generation Supervisor Agent will first delegate the article creation task to the Article Generation Agent
4. Then send the draft to the Article Reviewer Agent for detailed feedback
5. Return the draft with feedback to the Article Generation Agent for refinement
6. Repeat this cycle up to three times to continuously improve the article quality
7. Finally deliver the polished article to the user

This collaborative approach combines the strengths of all specialized agents - the researching capability of the Research Agent, the creative writing capabilities of the Article Generation Agent and the critical analysis skills of the Article Reviewer Agent - resulting in higher quality content than either agent could produce independently.

We can now set a dictionary with agents names for better visualization of the traces

In [ ]:
multi_agent_names = {
    f"{research_agent_id}/{research_agent_alias_id}": research_agent_name,
    f"{article_supervisor_agent_id}/{article_supervisor_agent_alias_id}": article_supervisor_agent_name,
    f"{interface_supervisor_agent_id}/{interface_supervisor_agent_alias_id}": interface_supervisor_agent_name
}

## Testing Agent
Let's test this multi-agent system by providing the Interface Supervisor Agent with mock news facts that have been collected by a journalist.

In [ ]:
news_facts = """NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
NeuraHealth CEO Marcus Williams stated the company invested $450 million in research and development over three years.
The platform will be piloted at five major hospital networks starting next month.
Senior Vice President of Product Development, Raj Patel, confirmed that FDA approval is expected by the third quarter.
Initial focus areas include cardiovascular disease, diabetes, and early cancer detection.
"""

In [ ]:
article_query_prompt = news_facts

article_query_prompt

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

interface_supervisor_agent_response = agents.invoke(
    article_query_prompt,
    interface_supervisor_agent_id,
    session_id=session_id,
    enable_trace=True,
    trace_level = "all",
    multi_agent_names=multi_agent_names
)

In [ ]:
print(interface_supervisor_agent_response)

# Summary

In this lab, we demonstrated how to implement a powerful **Multi-Agent Collaboration** pattern using Amazon Bedrock Agents. This pattern enables complex workflows through the orchestration of specialized agents working together to achieve superior results.

Our implementation consisted of five key components:

1. **Research Agent**: A specialized agent which can extract entities from news facts and provide detailed research on each entity to help write a balanced news article.

2. **Article Generation Agent**: A specialized content creator that produces high-quality articles following detailed journalistic guidelines across multiple dimensions including clarity, accuracy, engagement, structure, and technical excellence.

3. **Article Reviewer Agent**: A professional reviewer that analyzes content across multiple dimensions and provides detailed, constructive feedback with specific improvement suggestions.

4. **Article Supervisor Agent**: The orchestrator that coordinates the collaboration between the specialized agents, managing the iterative feedback loop to continuously refine the article quality.

5. **Interface Supervisor Agent**: This orchestrates the whole workflow of passing information from the user to the research and generation agents.

The multi-agent collaboration pattern demonstrated several key advantages:

- **Specialized Expertise**: Each agent focuses on its core competency - content creation or critical review
- **Iterative Improvement**: The feedback loop enables continuous refinement of the content
- **Workflow Automation**: The supervisor agent manages the entire process without requiring user intervention
- **Quality Enhancement**: The final output benefits from multiple perspectives and expertise

